In [68]:
import os

if "notebooks" in os.getcwd():
    %cd ..
from pathlib import Path

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns
from src.utils.defines import INTERIM_DATA_DIR, PROCESSED_DATA_DIR, RAW_DATA_DIR
import matplotlib.patches as mpatches

In [2]:
unlabeled_reddit = pd.read_csv(os.path.join(RAW_DATA_DIR, "reddit_1M_unlabelled.csv"))
unlabeled_GAB = pd.read_csv(os.path.join(RAW_DATA_DIR, "gab_1M_unlabelled.csv"))

In [6]:
unlabeled_reddit['key'] = unlabeled_reddit.index.astype(str) + 'reddit'
unlabeled_GAB['key'] = unlabeled_GAB.index.astype(str) + 'GAB'

In [11]:
# Calculate the number of pieces to split the dataframe into
n = 200000
num_pieces = math.ceil(len(unlabeled_reddit) / n)

# Split the dataframe into multiple smaller dataframes
dfs_unlabeled_reddit = [unlabeled_reddit.iloc[i*n:(i+1)*n] for i in range(num_pieces)]
dfs_unlabeled_GAB = [unlabeled_GAB.iloc[i*n:(i+1)*n] for i in range(num_pieces)]

In [13]:
# Save each of the smaller dataframes to separate files
for i, df in enumerate(dfs_unlabeled_reddit):
    df.to_csv(f'df_{i}reddit.csv', index=False)

for i, df in enumerate(dfs_unlabeled_GAB):
    df.to_csv(f'df_{i}GAB.csv', index=False)

In [135]:
######################################################################################################
# REFORMAT OUTPUT FROM UNLABELED - task a
task_a_unlabeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_a_10k.csv"))
unlabeled_data_10k = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_data_10k.csv"))

In [138]:
# create final new data
merged_df = task_a_unlabeled.merge(unlabeled_data_10k, left_index=True, right_index=True, how='inner')

merged_df['target_a'] = np.where(merged_df['0_x'] > merged_df['1'], '0', '1')

merged_df = merged_df.drop(['0_x', '1'], axis=1)
merged_df['text'] = merged_df['0_y']
merged_df = merged_df.drop(['0_y'], axis=1)

merged_df

,target_a,text
0,0,Don't forget that I AM supporting you at the h...
1,0,My guess is the restaurant is at least 30 year...
2,0,John Nolte: Hillary Clinton is Dumb [URL]
3,0,"Jackson Pollock with his Artwork, 1949"
4,0,This is shocking!! What #CA voters approved an...
...,...,...
9995,0,#Music #KatelynTarver [URL]
9996,0,"""We want people working at Amazon who want to ..."
9997,0,Another gold seller confirms Bitcoin owners tr...
9998,0,"[USER] Assange is not a genius, but by the sta..."


In [151]:
# take only sexist data
sexist_new = merged_df.loc[merged_df['target_a'] == '1']
sexist_new.to_csv(f'GAB_task_b.csv', index=False)

sexist_new = sexist_new.drop(['target_a'],axis=1)
sexist_new.to_csv("unlabeled_task_b_GAB.csv", index=False)
######################################################################################################

In [ ]:
merged_df.head()

In [ ]:
merged_df.head()

In [ ]:
pd.DataFrame(np.where(merged_df.target_a == '1')).transpose()

In [ ]:
unlabeled_data_10k['0'][267]
